In [34]:
env.unwrapped.road.network.all_side_lanes(env.observation_type.observer_vehicle.lane_index)

/u/shuhan/anaconda3/envs/rl_hw/lib/python3.12/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.observation_type to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.observation_type` for environment variables or `env.get_wrapper_attr('observation_type')` that will search the reminding wrappers.
  logger.warn(


[('0', '1', 0), ('0', '1', 1), ('0', '1', 2), ('0', '1', 3), ('0', '1', 4)]

In [70]:
%load_ext autoreload
%autoreload 2

from IPython.display import HTML
import tqdm
import gymnasium
import highway_env
from matplotlib import pyplot as plt
%matplotlib inline

env = gymnasium.make('highway-fast-v0', render_mode='rgb_array',
                     config={"lanes_count": 5})
env.reset()

import matplotlib.animation as animation

frames = []
for _ in tqdm.trange(1):
    action = env.unwrapped.action_type.actions_indexes["FASTER"]
    obs, reward, done, truncated, info = env.step(action)
    frames.append(env.render())

fig, ax = plt.subplots()
ani = animation.ArtistAnimation(fig, [[ax.imshow(frame)] for frame in frames], interval=200, blit=True, repeat_delay=1000)
plt.close(fig)  # Prevent the static image from displaying
ani.save('environment_steps.gif', writer='pillow')

HTML(ani.to_jshtml())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 1/1 [00:00<00:00, 13.83it/s]


In [71]:
import numpy as np

# abs_y = obs[:, :, 2]
lane_cnt = 5

lane_width = 1.0 / lane_cnt

abs_y = obs[..., 2].copy()
abs_y[1:] += abs_y[:1]
abs_y += lane_width / 2

lane_id = (abs_y / lane_width).astype(np.int32)

In [72]:
lane_id

array([1, 3, 3, 2, 4], dtype=int32)

# Load from file

In [170]:
import numpy as np

file_name = '/storage/Datasets/highway_env/highway_fast_v0_dqn_meta_action_5_lanes/rollouts_train/rollout_57775.npz'
data = np.load(file_name)
observations = data['observations']
actions = data['actions']

observations.shape, actions.shape

((30, 5, 5), (30,))

In [171]:
observations[0, :, 2]

array([0. , 0. , 0.8, 0.2, 0.4], dtype=float32)

In [172]:
import numpy as np

# abs_y = obs[:, :, 2]
lane_cnt = 5

lane_width = 1.0 / lane_cnt

abs_y = observations[..., 2].copy()
abs_y[:, 1:] += abs_y[:, :1]
abs_y += lane_width / 2

lane_id = (abs_y / lane_width).astype(np.int32)

In [173]:
ego_lane_id = lane_id[:, 0]

In [174]:
ego_lane_id

array([0, 0, 1, 2, 3, 3, 4, 4, 4, 4, 3, 2, 2, 2, 3, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 3, 3, 3, 3, 2], dtype=int32)

In [175]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/u/shuhan/projects/vla')

from src.auto_labeling.highway_env.lane_change import LaneChangeTaskSpec

# cfgs = {'lanes_count': 5, 'max_hop': 4, 'cot_index_mode': 'right_before_next_action'}
cfgs = {'lanes_count': 5, 'max_hop': 4, 'cot_index_mode': 'both'}

task_spec = LaneChangeTaskSpec(observations, actions, cfgs)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [176]:
task_spec.get_task_hop_info()

{'hop_lane_ids': [0, 1, 2, 3, 4], 'hop_indices': array([0, 1, 2, 3, 5])}

In [177]:
task_spec.get_goal_spec()

'Goal is to reach Lane 4. Need to go through path Lane 0 -> Lane 1 -> Lane 2 -> Lane 3 -> Lane 4.'

In [178]:
task_spec.get_ending_spec()

'Now at Goal Lane 4. Finished.'

In [179]:
actions[11]

np.int64(1)

In [180]:
ego_lane_id

array([0, 0, 1, 2, 3, 3, 4, 4, 4, 4, 3, 2, 2, 2, 3, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 3, 3, 3, 3, 2], dtype=int32)

In [181]:
task_spec.get_multi_step_cot_prompt()

{1: 'Now at Lane 0. Follow Lane 1 -> Lane 2 -> Lane 3 and reach Lane 4. Next is Lane 1. Action: turn right.',
 0: 'Now at Lane 0. Follow Lane 1 -> Lane 2 -> Lane 3 and reach Lane 4. Next is Lane 1. Action: turn right.',
 2: 'Now at Lane 1. Follow Lane 2 -> Lane 3 and reach Lane 4. Next is Lane 2. Action: turn right.',
 3: 'Now at Lane 2. Follow Lane 3 and reach Lane 4. Next is Lane 3. Action: turn right.',
 5: 'Now at Lane 3. Goal Reachable. Next is Lane 4. Action: turn right.',
 4: 'Now at Lane 3. Goal Reachable. Next is Lane 4. Action: turn right.'}